# 1 导包和数据读取

In [10]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
plt.rcParams.update({'figure.max_open_warning': 0})
import seaborn as sns

# modelling
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score,cross_val_predict,KFold
from sklearn.metrics import make_scorer,mean_squared_error
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.svm import LinearSVR, SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor,AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import PolynomialFeatures,MinMaxScaler,StandardScaler

In [11]:
#load_dataset
lane=pd.read_csv('./data/Lane.csv') # 车道数据
light_status=pd.read_csv('./data/Light_status.csv') # 路口灯态相位数据
entrance_road=pd.read_csv('./data/Entrance_road.csv') # 进口道数据
flow=pd.read_csv('./data/Flow.csv') # 路口车流量数据

In [12]:
lane.head()

,LANE_ID,ARM_ID,LANE_NAME,DIR
0,1,E,左,L
1,1,N,左,L
2,1,S,左,L
3,1,W,左,L
4,2,E,直,S


In [13]:
light_status.head()

,CYCLE_START_TIME,STAGE_START_TIME,STAGE_END_TIME,STAGE_LENGTH,GREEN_TIME,GREEN_FLASH_TIME,YELLOW_TIME,ALL_RED_TIME,CHANNELS,LANES,PHASES,LANE_FUNCS
0,2023-08-01 0:01:08,2023-08-01 00:01:08,2023-08-01 00:01:30,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13"
1,2023-08-01 0:02:18,2023-08-01 00:02:18,2023-08-01 00:02:40,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13"
2,2023-08-01 0:04:38,2023-08-01 00:04:38,2023-08-01 00:05:00,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13"
3,2023-08-01 0:05:48,2023-08-01 00:05:48,2023-08-01 00:06:10,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13"
4,2023-08-01 0:06:58,2023-08-01 00:06:58,2023-08-01 00:07:20,22,19,0,3,0,"11,2,3,10","1_2,1_3,3_2,1_4,3_3,3_4","3,5,10","11,13"


In [14]:
entrance_road.head()

,ARM_ID,ARM_NAME,DIR
0,E,东进口道,E
1,N,北进口道,N
2,S,南进口道,S
3,W,西进口道,W


In [15]:
flow.head()

,LANE_ID,ARM_ID,START_TIME,END_TIME,VOLUMN_5MIN
0,1,E,2023-08-01 00:05:00,2023-08-01 00:10:00,2
1,1,E,2023-08-01 00:10:00,2023-08-01 00:15:00,1
2,1,E,2023-08-01 00:15:00,2023-08-01 00:20:00,0
3,1,E,2023-08-01 00:20:00,2023-08-01 00:25:00,1
4,1,E,2023-08-01 00:25:00,2023-08-01 00:30:00,0
